In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [25]:
from sklearn.model_selection import train_test_split
train = pd.read_csv('train.csv',index_col='id')
train

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,20.630780,20.437920,20.050756,20.020974,19.959646,20.948519,20.726709,20.311562,...,20.836400,20.536338,20.126561,20.102815,19.796722,20.783295,20.326879,20.014046,20.061940,19.862513
199987,GALAXY,362,25.283695,22.807243,21.069792,20.187531,19.595598,26.677671,22.818723,20.786120,...,28.895442,22.250489,20.091693,18.985791,18.803354,26.921903,21.952440,19.776321,18.990039,18.528306
199988,STAR_RED_DWARF,146,23.609826,21.902709,20.330747,18.457535,17.326771,24.320816,22.287946,20.711737,...,25.982378,22.132993,20.329624,18.507337,17.418521,24.503460,21.897599,20.420524,18.443126,17.331724


Catboost는 parameter를 설정하지 않기 때문에 이를 돌린다.
Scaler중에서 뭐가 적합한지 보기 위해 4가지 Scaler를 쓰는데 동일한 조건으로 catboost를 돌려서 log loss를 비교하고 가장 작은 값을 도출하는 Scaler를 선택한다.
FiberID는 제외하고 Scaling한 후에 다시 fiberID를 합친다.

In [9]:
# RobustScaler
from sklearn.preprocessing import RobustScaler
train_for_scaling = train.iloc[:,2:] #type과 fiberID 제외하고 데이터 생성
robustScaler = RobustScaler()
print(robustScaler.fit(train_for_scaling))
train_robustscaled = robustScaler.transform(train_for_scaling) #robustscaling 적용
train_robustscaled = pd.DataFrame(train_robustscaled,columns=train_for_scaling.columns)
train_robustscaled = pd.concat([train[['type', 'fiberID']], train_robustscaled], axis=1) 
train_robustscaled #robustscaling과 type,fiberID 합치기

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)


,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,601,0.927836,0.623783,0.753702,0.916931,1.041252,0.639184,0.634486,0.805768,...,0.838327,0.638332,0.770905,0.825931,0.748549,0.859066,0.664478,0.744135,0.795289,0.978342
1,QSO,788,0.223298,0.328228,0.496221,0.713509,0.757221,0.341950,0.386929,0.531161,...,0.389795,0.412532,0.573234,0.655629,0.823214,0.398328,0.419522,0.539673,0.685895,0.774402
2,QSO,427,-1.204186,-1.296932,-1.124607,-1.010056,-0.801392,-1.200267,-1.222701,-1.046600,...,-0.980354,-0.985231,-0.837531,-0.656544,-0.455081,-0.949423,-0.987518,-0.838240,-0.662475,-0.453490
3,QSO,864,-0.032482,0.055586,0.191466,0.343934,0.346375,-0.004199,0.099931,0.233233,...,0.026301,0.140866,0.269585,0.356218,0.426423,0.133448,0.157305,0.250155,0.373217,0.466269
4,STAR_RED_DWARF,612,2.226460,1.288028,0.817437,0.198234,-0.090595,1.940455,1.433796,0.839643,...,2.154141,0.982669,0.836473,0.308756,0.067117,1.639432,1.247397,0.781377,0.282689,0.088207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,-0.095930,0.217909,0.241661,0.420093,0.548298,-0.041957,0.264872,0.270482,...,0.184161,0.328591,0.316947,0.474749,0.540180,0.138063,0.269970,0.291783,0.474791,0.565716
199987,GALAXY,362,1.759416,1.185329,0.654667,0.491704,0.400205,2.348040,1.107414,0.459206,...,3.344960,0.921627,0.305237,0.098497,0.209382,2.389373,0.833175,0.212136,0.116437,0.137232
199988,STAR_RED_DWARF,146,1.091962,0.815997,0.355138,-0.252104,-0.522744,1.364844,0.893648,0.429626,...,2.202441,0.880978,0.385142,-0.062662,-0.251776,1.502419,0.814175,0.427970,-0.066405,-0.247054
199989,GALAXY,85,0.119882,0.268249,0.233959,0.322353,0.377780,0.273255,0.161383,0.095711,...,-0.582843,-0.526213,-0.564692,-0.509540,-0.332802,-0.475946,-0.534496,-0.570515,-0.484095,-0.363270


In [10]:
X_robustscaled = train_robustscaled.drop(['type'], axis = 1)
y_robustscaled = train_robustscaled['type']
X_train_robustscaled, X_test_robustscaled, y_train_robustscaled, y_test_robustscaled = train_test_split(X_robustscaled, y_robustscaled, test_size= 0.2, random_state=42)

In [34]:
#Catboost
import catboost as cb
cb_model = cb.CatBoostClassifier()
cb_model.fit(X_train_robustscaled,y_train_robustscaled)

0:	learn: 2.6656863	total: 390ms	remaining: 6m 29s
1:	learn: 2.4753789	total: 715ms	remaining: 5m 56s
2:	learn: 2.3315841	total: 1.04s	remaining: 5m 45s
3:	learn: 2.2134448	total: 1.36s	remaining: 5m 38s
4:	learn: 2.1142810	total: 1.69s	remaining: 5m 35s
5:	learn: 2.0309055	total: 2.01s	remaining: 5m 32s
6:	learn: 1.9554229	total: 2.32s	remaining: 5m 29s
7:	learn: 1.8859912	total: 2.65s	remaining: 5m 28s
8:	learn: 1.8263944	total: 2.97s	remaining: 5m 27s
9:	learn: 1.7712744	total: 3.29s	remaining: 5m 26s
10:	learn: 1.7207625	total: 3.61s	remaining: 5m 24s
11:	learn: 1.6740573	total: 3.93s	remaining: 5m 23s
12:	learn: 1.6308702	total: 4.25s	remaining: 5m 22s
13:	learn: 1.5919404	total: 4.57s	remaining: 5m 22s
14:	learn: 1.5553841	total: 4.9s	remaining: 5m 21s
15:	learn: 1.5202630	total: 5.22s	remaining: 5m 21s
16:	learn: 1.4885246	total: 5.54s	remaining: 5m 20s
17:	learn: 1.4583453	total: 5.87s	remaining: 5m 20s
18:	learn: 1.4287061	total: 6.18s	remaining: 5m 19s
19:	learn: 1.4017088	to

158:	learn: 0.6177756	total: 50.3s	remaining: 4m 26s
159:	learn: 0.6166163	total: 50.6s	remaining: 4m 25s
160:	learn: 0.6151816	total: 51s	remaining: 4m 25s
161:	learn: 0.6136948	total: 51.3s	remaining: 4m 25s
162:	learn: 0.6121909	total: 51.6s	remaining: 4m 24s
163:	learn: 0.6108295	total: 51.9s	remaining: 4m 24s
164:	learn: 0.6102258	total: 52.2s	remaining: 4m 24s
165:	learn: 0.6087125	total: 52.5s	remaining: 4m 23s
166:	learn: 0.6072965	total: 52.9s	remaining: 4m 23s
167:	learn: 0.6057410	total: 53.2s	remaining: 4m 23s
168:	learn: 0.6042011	total: 53.5s	remaining: 4m 23s
169:	learn: 0.6030472	total: 53.8s	remaining: 4m 22s
170:	learn: 0.6018268	total: 54.1s	remaining: 4m 22s
171:	learn: 0.6009625	total: 54.4s	remaining: 4m 22s
172:	learn: 0.6001092	total: 54.7s	remaining: 4m 21s
173:	learn: 0.5989977	total: 55.1s	remaining: 4m 21s
174:	learn: 0.5984477	total: 55.4s	remaining: 4m 21s
175:	learn: 0.5974221	total: 55.7s	remaining: 4m 20s
176:	learn: 0.5964103	total: 56s	remaining: 4m 2

312:	learn: 0.5103514	total: 1m 38s	remaining: 3m 36s
313:	learn: 0.5099516	total: 1m 39s	remaining: 3m 36s
314:	learn: 0.5095012	total: 1m 39s	remaining: 3m 36s
315:	learn: 0.5088053	total: 1m 39s	remaining: 3m 35s
316:	learn: 0.5083351	total: 1m 39s	remaining: 3m 35s
317:	learn: 0.5078769	total: 1m 40s	remaining: 3m 35s
318:	learn: 0.5076111	total: 1m 40s	remaining: 3m 34s
319:	learn: 0.5074577	total: 1m 40s	remaining: 3m 34s
320:	learn: 0.5072572	total: 1m 41s	remaining: 3m 34s
321:	learn: 0.5068528	total: 1m 41s	remaining: 3m 33s
322:	learn: 0.5065881	total: 1m 41s	remaining: 3m 33s
323:	learn: 0.5061643	total: 1m 42s	remaining: 3m 33s
324:	learn: 0.5057767	total: 1m 42s	remaining: 3m 32s
325:	learn: 0.5053746	total: 1m 42s	remaining: 3m 32s
326:	learn: 0.5048436	total: 1m 43s	remaining: 3m 32s
327:	learn: 0.5046277	total: 1m 43s	remaining: 3m 31s
328:	learn: 0.5040872	total: 1m 43s	remaining: 3m 31s
329:	learn: 0.5039179	total: 1m 44s	remaining: 3m 31s
330:	learn: 0.5034861	total:

466:	learn: 0.4667489	total: 2m 27s	remaining: 2m 48s
467:	learn: 0.4666436	total: 2m 27s	remaining: 2m 48s
468:	learn: 0.4665804	total: 2m 28s	remaining: 2m 47s
469:	learn: 0.4665260	total: 2m 28s	remaining: 2m 47s
470:	learn: 0.4663773	total: 2m 28s	remaining: 2m 47s
471:	learn: 0.4662921	total: 2m 29s	remaining: 2m 46s
472:	learn: 0.4662247	total: 2m 29s	remaining: 2m 46s
473:	learn: 0.4659666	total: 2m 29s	remaining: 2m 46s
474:	learn: 0.4656193	total: 2m 30s	remaining: 2m 46s
475:	learn: 0.4652686	total: 2m 30s	remaining: 2m 45s
476:	learn: 0.4650218	total: 2m 30s	remaining: 2m 45s
477:	learn: 0.4647138	total: 2m 31s	remaining: 2m 45s
478:	learn: 0.4644765	total: 2m 31s	remaining: 2m 44s
479:	learn: 0.4643703	total: 2m 31s	remaining: 2m 44s
480:	learn: 0.4640412	total: 2m 32s	remaining: 2m 44s
481:	learn: 0.4639111	total: 2m 32s	remaining: 2m 43s
482:	learn: 0.4637249	total: 2m 32s	remaining: 2m 43s
483:	learn: 0.4635405	total: 2m 33s	remaining: 2m 43s
484:	learn: 0.4632968	total:

620:	learn: 0.4414353	total: 3m 16s	remaining: 1m 59s
621:	learn: 0.4412639	total: 3m 16s	remaining: 1m 59s
622:	learn: 0.4410775	total: 3m 16s	remaining: 1m 59s
623:	learn: 0.4409945	total: 3m 17s	remaining: 1m 58s
624:	learn: 0.4409395	total: 3m 17s	remaining: 1m 58s
625:	learn: 0.4408902	total: 3m 17s	remaining: 1m 58s
626:	learn: 0.4406803	total: 3m 18s	remaining: 1m 57s
627:	learn: 0.4405899	total: 3m 18s	remaining: 1m 57s
628:	learn: 0.4404628	total: 3m 18s	remaining: 1m 57s
629:	learn: 0.4403578	total: 3m 19s	remaining: 1m 56s
630:	learn: 0.4402817	total: 3m 19s	remaining: 1m 56s
631:	learn: 0.4402296	total: 3m 19s	remaining: 1m 56s
632:	learn: 0.4401503	total: 3m 19s	remaining: 1m 55s
633:	learn: 0.4399692	total: 3m 20s	remaining: 1m 55s
634:	learn: 0.4399127	total: 3m 20s	remaining: 1m 55s
635:	learn: 0.4398088	total: 3m 20s	remaining: 1m 55s
636:	learn: 0.4397700	total: 3m 21s	remaining: 1m 54s
637:	learn: 0.4396569	total: 3m 21s	remaining: 1m 54s
638:	learn: 0.4394840	total:

773:	learn: 0.4239521	total: 4m 4s	remaining: 1m 11s
774:	learn: 0.4239075	total: 4m 4s	remaining: 1m 11s
775:	learn: 0.4237592	total: 4m 4s	remaining: 1m 10s
776:	learn: 0.4236281	total: 4m 5s	remaining: 1m 10s
777:	learn: 0.4235748	total: 4m 5s	remaining: 1m 10s
778:	learn: 0.4235305	total: 4m 5s	remaining: 1m 9s
779:	learn: 0.4234782	total: 4m 6s	remaining: 1m 9s
780:	learn: 0.4233927	total: 4m 6s	remaining: 1m 9s
781:	learn: 0.4233213	total: 4m 6s	remaining: 1m 8s
782:	learn: 0.4232389	total: 4m 7s	remaining: 1m 8s
783:	learn: 0.4230890	total: 4m 7s	remaining: 1m 8s
784:	learn: 0.4230434	total: 4m 7s	remaining: 1m 7s
785:	learn: 0.4230092	total: 4m 8s	remaining: 1m 7s
786:	learn: 0.4229096	total: 4m 8s	remaining: 1m 7s
787:	learn: 0.4228650	total: 4m 8s	remaining: 1m 6s
788:	learn: 0.4227755	total: 4m 9s	remaining: 1m 6s
789:	learn: 0.4226248	total: 4m 9s	remaining: 1m 6s
790:	learn: 0.4225436	total: 4m 9s	remaining: 1m 5s
791:	learn: 0.4225168	total: 4m 9s	remaining: 1m 5s
792:	le

929:	learn: 0.4120702	total: 4m 53s	remaining: 22.1s
930:	learn: 0.4119822	total: 4m 53s	remaining: 21.8s
931:	learn: 0.4119471	total: 4m 53s	remaining: 21.4s
932:	learn: 0.4118987	total: 4m 54s	remaining: 21.1s
933:	learn: 0.4118244	total: 4m 54s	remaining: 20.8s
934:	learn: 0.4117817	total: 4m 54s	remaining: 20.5s
935:	learn: 0.4116800	total: 4m 55s	remaining: 20.2s
936:	learn: 0.4116169	total: 4m 55s	remaining: 19.9s
937:	learn: 0.4115068	total: 4m 55s	remaining: 19.6s
938:	learn: 0.4113840	total: 4m 56s	remaining: 19.2s
939:	learn: 0.4112982	total: 4m 56s	remaining: 18.9s
940:	learn: 0.4112610	total: 4m 56s	remaining: 18.6s
941:	learn: 0.4111567	total: 4m 57s	remaining: 18.3s
942:	learn: 0.4111337	total: 4m 57s	remaining: 18s
943:	learn: 0.4110358	total: 4m 57s	remaining: 17.7s
944:	learn: 0.4109312	total: 4m 57s	remaining: 17.3s
945:	learn: 0.4108902	total: 4m 58s	remaining: 17s
946:	learn: 0.4107665	total: 4m 58s	remaining: 16.7s
947:	learn: 0.4107224	total: 4m 58s	remaining: 16.

In [35]:
cb_model.score(X_test_robustscaled,y_test_robustscaled)

0.8579964499112478

In [37]:
from sklearn.metrics import log_loss
y_pred_prob_cb_robustscaled = cb_model.predict_proba(X_test_robustscaled)
log_loss(y_true=y_test_robustscaled, y_pred=y_pred_prob_cb_robustscaled)

0.42774662154063664

In [48]:
# StandardScaler
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
print(standardScaler.fit(train_for_scaling))
train_standardscaled = standardScaler.transform(train_for_scaling)
train_standardscaled = pd.DataFrame(train_standardscaled,columns=train_for_scaling.columns)
train_standardscaled = pd.concat([train[['type', 'fiberID']], train_standardscaled], axis=1) 
train_standardscaled #standardscaling과 type,fiberID 합치기

StandardScaler(copy=True, with_mean=True, with_std=True)


,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,601,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,0.028019,...,0.000844,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607
1,QSO,788,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,0.020351,...,-0.000604,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857
2,QSO,427,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,-0.023707,...,-0.005030,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743
3,QSO,864,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,0.012031,...,-0.001778,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680
4,STAR_RED_DWARF,612,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,0.028965,...,0.005094,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,0.002305,0.011340,0.012975,0.016956,0.018557,0.002421,0.002208,0.013072,...,-0.001269,0.013488,0.016917,0.016580,0.014700,0.005497,0.011022,0.013677,0.018063,0.019997
199987,GALAXY,362,0.002697,0.026585,0.020991,0.018385,0.015615,0.003794,0.005000,0.018342,...,0.008940,0.024592,0.016559,0.008915,0.007738,0.055691,0.021073,0.011903,0.009892,0.010016
199988,STAR_RED_DWARF,146,0.002556,0.020765,0.015178,0.003550,-0.002722,0.003229,0.004291,0.017516,...,0.005250,0.023831,0.019006,0.005632,-0.001967,0.035916,0.020734,0.016711,0.005723,0.001066
199989,GALAXY,85,0.002351,0.012134,0.012826,0.015007,0.015169,0.002602,0.001865,0.008191,...,-0.003746,-0.002517,-0.010080,-0.003472,-0.003672,-0.008192,-0.003335,-0.005532,-0.003801,-0.001641


In [41]:
X_standardscaled = train_standardscaled.drop(['type'], axis = 1)
y_standardscaled = train_standardscaled['type']
X_train_standardscaled, X_test_standardscaled, y_train_standardscaled, y_test_standardscaled = train_test_split(X_standardscaled, y_standardscaled, test_size= 0.2, random_state=42)

In [43]:
#Catboost
cb_model.fit(X_train_standardscaled,y_train_standardscaled)

0:	learn: 2.6656863	total: 327ms	remaining: 5m 26s
1:	learn: 2.4753789	total: 647ms	remaining: 5m 22s
2:	learn: 2.3315841	total: 965ms	remaining: 5m 20s
3:	learn: 2.2134448	total: 1.29s	remaining: 5m 21s
4:	learn: 2.1142810	total: 1.61s	remaining: 5m 20s
5:	learn: 2.0309055	total: 1.93s	remaining: 5m 20s
6:	learn: 1.9554234	total: 2.26s	remaining: 5m 20s
7:	learn: 1.8859903	total: 2.58s	remaining: 5m 20s
8:	learn: 1.8263935	total: 2.9s	remaining: 5m 19s
9:	learn: 1.7712735	total: 3.23s	remaining: 5m 20s
10:	learn: 1.7207616	total: 3.55s	remaining: 5m 19s
11:	learn: 1.6740564	total: 3.86s	remaining: 5m 18s
12:	learn: 1.6308694	total: 4.19s	remaining: 5m 17s
13:	learn: 1.5919395	total: 4.5s	remaining: 5m 17s
14:	learn: 1.5553836	total: 4.83s	remaining: 5m 17s
15:	learn: 1.5202624	total: 5.15s	remaining: 5m 16s
16:	learn: 1.4885240	total: 5.47s	remaining: 5m 16s
17:	learn: 1.4583447	total: 5.79s	remaining: 5m 15s
18:	learn: 1.4287055	total: 6.12s	remaining: 5m 15s
19:	learn: 1.4017082	tot

158:	learn: 0.6177123	total: 50.4s	remaining: 4m 26s
159:	learn: 0.6170090	total: 50.7s	remaining: 4m 26s
160:	learn: 0.6155450	total: 51s	remaining: 4m 25s
161:	learn: 0.6141119	total: 51.4s	remaining: 4m 25s
162:	learn: 0.6129447	total: 51.7s	remaining: 4m 25s
163:	learn: 0.6115764	total: 52s	remaining: 4m 25s
164:	learn: 0.6109765	total: 52.3s	remaining: 4m 24s
165:	learn: 0.6094568	total: 52.6s	remaining: 4m 24s
166:	learn: 0.6078010	total: 53s	remaining: 4m 24s
167:	learn: 0.6062276	total: 53.3s	remaining: 4m 23s
168:	learn: 0.6046999	total: 53.6s	remaining: 4m 23s
169:	learn: 0.6035445	total: 53.9s	remaining: 4m 23s
170:	learn: 0.6023334	total: 54.3s	remaining: 4m 23s
171:	learn: 0.6014730	total: 54.6s	remaining: 4m 22s
172:	learn: 0.6004369	total: 54.9s	remaining: 4m 22s
173:	learn: 0.5993360	total: 55.2s	remaining: 4m 22s
174:	learn: 0.5986939	total: 55.5s	remaining: 4m 21s
175:	learn: 0.5981715	total: 55.8s	remaining: 4m 21s
176:	learn: 0.5971596	total: 56.2s	remaining: 4m 21s

312:	learn: 0.5102284	total: 1m 39s	remaining: 3m 37s
313:	learn: 0.5097975	total: 1m 39s	remaining: 3m 37s
314:	learn: 0.5092616	total: 1m 39s	remaining: 3m 37s
315:	learn: 0.5085752	total: 1m 40s	remaining: 3m 36s
316:	learn: 0.5081052	total: 1m 40s	remaining: 3m 36s
317:	learn: 0.5076315	total: 1m 40s	remaining: 3m 36s
318:	learn: 0.5073610	total: 1m 41s	remaining: 3m 36s
319:	learn: 0.5069203	total: 1m 41s	remaining: 3m 35s
320:	learn: 0.5067215	total: 1m 41s	remaining: 3m 35s
321:	learn: 0.5063246	total: 1m 42s	remaining: 3m 35s
322:	learn: 0.5060597	total: 1m 42s	remaining: 3m 34s
323:	learn: 0.5056114	total: 1m 42s	remaining: 3m 34s
324:	learn: 0.5052348	total: 1m 43s	remaining: 3m 34s
325:	learn: 0.5047967	total: 1m 43s	remaining: 3m 33s
326:	learn: 0.5042816	total: 1m 43s	remaining: 3m 33s
327:	learn: 0.5040602	total: 1m 44s	remaining: 3m 33s
328:	learn: 0.5035851	total: 1m 44s	remaining: 3m 32s
329:	learn: 0.5034144	total: 1m 44s	remaining: 3m 32s
330:	learn: 0.5029729	total:

466:	learn: 0.4667797	total: 2m 28s	remaining: 2m 49s
467:	learn: 0.4665541	total: 2m 28s	remaining: 2m 48s
468:	learn: 0.4664244	total: 2m 28s	remaining: 2m 48s
469:	learn: 0.4663698	total: 2m 29s	remaining: 2m 48s
470:	learn: 0.4662170	total: 2m 29s	remaining: 2m 47s
471:	learn: 0.4659856	total: 2m 29s	remaining: 2m 47s
472:	learn: 0.4659150	total: 2m 30s	remaining: 2m 47s
473:	learn: 0.4656163	total: 2m 30s	remaining: 2m 46s
474:	learn: 0.4653391	total: 2m 30s	remaining: 2m 46s
475:	learn: 0.4649779	total: 2m 31s	remaining: 2m 46s
476:	learn: 0.4647237	total: 2m 31s	remaining: 2m 46s
477:	learn: 0.4645261	total: 2m 31s	remaining: 2m 45s
478:	learn: 0.4642912	total: 2m 32s	remaining: 2m 45s
479:	learn: 0.4641865	total: 2m 32s	remaining: 2m 45s
480:	learn: 0.4638928	total: 2m 32s	remaining: 2m 44s
481:	learn: 0.4637653	total: 2m 32s	remaining: 2m 44s
482:	learn: 0.4635724	total: 2m 33s	remaining: 2m 44s
483:	learn: 0.4633175	total: 2m 33s	remaining: 2m 43s
484:	learn: 0.4630932	total:

620:	learn: 0.4410376	total: 3m 16s	remaining: 2m
621:	learn: 0.4408762	total: 3m 17s	remaining: 1m 59s
622:	learn: 0.4406934	total: 3m 17s	remaining: 1m 59s
623:	learn: 0.4406105	total: 3m 17s	remaining: 1m 59s
624:	learn: 0.4405573	total: 3m 18s	remaining: 1m 58s
625:	learn: 0.4405072	total: 3m 18s	remaining: 1m 58s
626:	learn: 0.4403040	total: 3m 18s	remaining: 1m 58s
627:	learn: 0.4402085	total: 3m 19s	remaining: 1m 57s
628:	learn: 0.4400765	total: 3m 19s	remaining: 1m 57s
629:	learn: 0.4399477	total: 3m 19s	remaining: 1m 57s
630:	learn: 0.4398705	total: 3m 20s	remaining: 1m 57s
631:	learn: 0.4398225	total: 3m 20s	remaining: 1m 56s
632:	learn: 0.4396862	total: 3m 20s	remaining: 1m 56s
633:	learn: 0.4396350	total: 3m 21s	remaining: 1m 56s
634:	learn: 0.4395839	total: 3m 21s	remaining: 1m 55s
635:	learn: 0.4394701	total: 3m 21s	remaining: 1m 55s
636:	learn: 0.4394326	total: 3m 21s	remaining: 1m 55s
637:	learn: 0.4393245	total: 3m 22s	remaining: 1m 54s
638:	learn: 0.4391650	total: 3m 

773:	learn: 0.4239483	total: 4m 5s	remaining: 1m 11s
774:	learn: 0.4238294	total: 4m 5s	remaining: 1m 11s
775:	learn: 0.4236795	total: 4m 6s	remaining: 1m 11s
776:	learn: 0.4235472	total: 4m 6s	remaining: 1m 10s
777:	learn: 0.4234596	total: 4m 6s	remaining: 1m 10s
778:	learn: 0.4234139	total: 4m 7s	remaining: 1m 10s
779:	learn: 0.4233454	total: 4m 7s	remaining: 1m 9s
780:	learn: 0.4232555	total: 4m 7s	remaining: 1m 9s
781:	learn: 0.4231988	total: 4m 8s	remaining: 1m 9s
782:	learn: 0.4231160	total: 4m 8s	remaining: 1m 8s
783:	learn: 0.4229682	total: 4m 8s	remaining: 1m 8s
784:	learn: 0.4229209	total: 4m 8s	remaining: 1m 8s
785:	learn: 0.4228864	total: 4m 9s	remaining: 1m 7s
786:	learn: 0.4227774	total: 4m 9s	remaining: 1m 7s
787:	learn: 0.4227316	total: 4m 9s	remaining: 1m 7s
788:	learn: 0.4225705	total: 4m 10s	remaining: 1m 6s
789:	learn: 0.4224197	total: 4m 10s	remaining: 1m 6s
790:	learn: 0.4223355	total: 4m 10s	remaining: 1m 6s
791:	learn: 0.4223081	total: 4m 11s	remaining: 1m 5s
79

929:	learn: 0.4118970	total: 4m 54s	remaining: 22.2s
930:	learn: 0.4118087	total: 4m 55s	remaining: 21.9s
931:	learn: 0.4117704	total: 4m 55s	remaining: 21.6s
932:	learn: 0.4117295	total: 4m 55s	remaining: 21.2s
933:	learn: 0.4116933	total: 4m 56s	remaining: 20.9s
934:	learn: 0.4116502	total: 4m 56s	remaining: 20.6s
935:	learn: 0.4115723	total: 4m 56s	remaining: 20.3s
936:	learn: 0.4115078	total: 4m 56s	remaining: 20s
937:	learn: 0.4113966	total: 4m 57s	remaining: 19.6s
938:	learn: 0.4112748	total: 4m 57s	remaining: 19.3s
939:	learn: 0.4111872	total: 4m 57s	remaining: 19s
940:	learn: 0.4110949	total: 4m 58s	remaining: 18.7s
941:	learn: 0.4109775	total: 4m 58s	remaining: 18.4s
942:	learn: 0.4108982	total: 4m 58s	remaining: 18.1s
943:	learn: 0.4108755	total: 4m 59s	remaining: 17.7s
944:	learn: 0.4107674	total: 4m 59s	remaining: 17.4s
945:	learn: 0.4107280	total: 4m 59s	remaining: 17.1s
946:	learn: 0.4106058	total: 5m	remaining: 16.8s
947:	learn: 0.4105611	total: 5m	remaining: 16.5s
948:	

In [44]:
cb_model.score(X_test_standardscaled,y_test_standardscaled)

0.8580964524113103

In [45]:
y_pred_prob_cb_standardscaled = cb_model.predict_proba(X_test_standardscaled)
log_loss(y_true=y_test_standardscaled, y_pred=y_pred_prob_cb_standardscaled)

0.4278871022373594

In [47]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
print(minmaxscaler.fit(train_for_scaling))
train_minmaxscaled = minmaxscaler.transform(train_for_scaling)
train_minmaxscaled = pd.DataFrame(train_minmaxscaled,columns=train_for_scaling.columns)
train_minmaxscaled = pd.concat([train[['type', 'fiberID']], train_minmaxscaled], axis=1) 
train_minmaxscaled #minmaxscaling과 type,fiberID 합치기

MinMaxScaler(copy=True, feature_range=(0, 1))


,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,601,0.996482,0.919251,0.899884,0.846694,0.717526,0.997407,0.465351,0.644025,...,0.075753,0.681920,0.774434,0.805796,0.633838,0.644799,0.777599,0.749031,0.906082,0.778933
1,QSO,788,0.996481,0.919234,0.899863,0.846679,0.717506,0.997407,0.465350,0.644004,...,0.075749,0.681903,0.774415,0.805786,0.633843,0.644768,0.777584,0.749018,0.906074,0.778914
2,QSO,427,0.996481,0.919143,0.899731,0.846551,0.717396,0.997405,0.465341,0.643887,...,0.075739,0.681797,0.774279,0.805709,0.633762,0.644678,0.777499,0.748934,0.905981,0.778800
3,QSO,864,0.996481,0.919219,0.899838,0.846652,0.717477,0.997406,0.465348,0.643982,...,0.075746,0.681882,0.774386,0.805768,0.633818,0.644750,0.777568,0.749000,0.906052,0.778885
4,STAR_RED_DWARF,612,0.996482,0.919288,0.899889,0.846641,0.717446,0.997409,0.465356,0.644027,...,0.075763,0.681946,0.774440,0.805766,0.633795,0.644851,0.777634,0.749033,0.906046,0.778850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,0.996481,0.919228,0.899842,0.846657,0.717491,0.997406,0.465349,0.643985,...,0.075748,0.681897,0.774390,0.805775,0.633825,0.644751,0.777575,0.749003,0.906060,0.778895
199987,GALAXY,362,0.996482,0.919282,0.899876,0.846663,0.717481,0.997409,0.465354,0.643999,...,0.075773,0.681942,0.774389,0.805753,0.633804,0.644901,0.777609,0.748998,0.906035,0.778855
199988,STAR_RED_DWARF,146,0.996482,0.919262,0.899852,0.846607,0.717415,0.997408,0.465353,0.643997,...,0.075764,0.681939,0.774397,0.805744,0.633775,0.644842,0.777608,0.749011,0.906022,0.778819
199989,GALAXY,85,0.996481,0.919231,0.899842,0.846650,0.717479,0.997407,0.465349,0.643972,...,0.075742,0.681832,0.774305,0.805717,0.633770,0.644710,0.777526,0.748950,0.905993,0.778808


In [50]:
X_minmaxscaled = train_minmaxscaled.drop(['type'], axis = 1)
y_minmaxscaled = train_minmaxscaled['type']
X_train_minmaxscaled, X_test_minmaxscaled, y_train_minmaxscaled, y_test_minmaxscaled = train_test_split(X_minmaxscaled, y_minmaxscaled, test_size= 0.2, random_state=42)

In [51]:
#Catboost
cb_model.fit(X_train_minmaxscaled,y_train_minmaxscaled)

0:	learn: 2.6627966	total: 327ms	remaining: 5m 26s
1:	learn: 2.4710353	total: 652ms	remaining: 5m 25s
2:	learn: 2.3310804	total: 967ms	remaining: 5m 21s
3:	learn: 2.2140082	total: 1.28s	remaining: 5m 19s
4:	learn: 2.1139599	total: 1.6s	remaining: 5m 17s
5:	learn: 2.0278627	total: 1.91s	remaining: 5m 16s
6:	learn: 1.9529225	total: 2.24s	remaining: 5m 17s
7:	learn: 1.8857743	total: 2.56s	remaining: 5m 17s
8:	learn: 1.8252872	total: 2.88s	remaining: 5m 17s
9:	learn: 1.7700156	total: 3.21s	remaining: 5m 18s
10:	learn: 1.7211913	total: 3.54s	remaining: 5m 18s
11:	learn: 1.6743562	total: 3.86s	remaining: 5m 18s
12:	learn: 1.6322169	total: 4.18s	remaining: 5m 17s
13:	learn: 1.5939974	total: 4.5s	remaining: 5m 16s
14:	learn: 1.5573708	total: 4.82s	remaining: 5m 16s
15:	learn: 1.5208727	total: 5.13s	remaining: 5m 15s
16:	learn: 1.4887937	total: 5.45s	remaining: 5m 15s
17:	learn: 1.4582885	total: 5.76s	remaining: 5m 14s
18:	learn: 1.4270702	total: 6.08s	remaining: 5m 13s
19:	learn: 1.3983123	tot

158:	learn: 0.6167976	total: 50s	remaining: 4m 24s
159:	learn: 0.6155720	total: 50.3s	remaining: 4m 23s
160:	learn: 0.6138822	total: 50.6s	remaining: 4m 23s
161:	learn: 0.6124197	total: 50.9s	remaining: 4m 23s
162:	learn: 0.6111916	total: 51.2s	remaining: 4m 23s
163:	learn: 0.6102504	total: 51.6s	remaining: 4m 22s
164:	learn: 0.6095782	total: 51.9s	remaining: 4m 22s
165:	learn: 0.6086910	total: 52.2s	remaining: 4m 22s
166:	learn: 0.6071456	total: 52.5s	remaining: 4m 21s
167:	learn: 0.6054646	total: 52.8s	remaining: 4m 21s
168:	learn: 0.6041114	total: 53.1s	remaining: 4m 21s
169:	learn: 0.6029379	total: 53.4s	remaining: 4m 20s
170:	learn: 0.6017320	total: 53.8s	remaining: 4m 20s
171:	learn: 0.6009886	total: 54.1s	remaining: 4m 20s
172:	learn: 0.6002197	total: 54.4s	remaining: 4m 20s
173:	learn: 0.5994879	total: 54.7s	remaining: 4m 19s
174:	learn: 0.5990309	total: 55s	remaining: 4m 19s
175:	learn: 0.5985897	total: 55.3s	remaining: 4m 19s
176:	learn: 0.5976573	total: 55.6s	remaining: 4m 1

312:	learn: 0.5105012	total: 1m 38s	remaining: 3m 35s
313:	learn: 0.5101277	total: 1m 38s	remaining: 3m 35s
314:	learn: 0.5096500	total: 1m 38s	remaining: 3m 35s
315:	learn: 0.5089302	total: 1m 39s	remaining: 3m 34s
316:	learn: 0.5084141	total: 1m 39s	remaining: 3m 34s
317:	learn: 0.5080493	total: 1m 39s	remaining: 3m 34s
318:	learn: 0.5075319	total: 1m 40s	remaining: 3m 34s
319:	learn: 0.5074035	total: 1m 40s	remaining: 3m 33s
320:	learn: 0.5072117	total: 1m 40s	remaining: 3m 33s
321:	learn: 0.5068027	total: 1m 41s	remaining: 3m 33s
322:	learn: 0.5065180	total: 1m 41s	remaining: 3m 32s
323:	learn: 0.5059675	total: 1m 41s	remaining: 3m 32s
324:	learn: 0.5058212	total: 1m 42s	remaining: 3m 32s
325:	learn: 0.5054046	total: 1m 42s	remaining: 3m 31s
326:	learn: 0.5049139	total: 1m 42s	remaining: 3m 31s
327:	learn: 0.5047531	total: 1m 43s	remaining: 3m 31s
328:	learn: 0.5042639	total: 1m 43s	remaining: 3m 30s
329:	learn: 0.5040949	total: 1m 43s	remaining: 3m 30s
330:	learn: 0.5038383	total:

466:	learn: 0.4679132	total: 2m 26s	remaining: 2m 47s
467:	learn: 0.4675995	total: 2m 27s	remaining: 2m 47s
468:	learn: 0.4675381	total: 2m 27s	remaining: 2m 46s
469:	learn: 0.4674655	total: 2m 27s	remaining: 2m 46s
470:	learn: 0.4672937	total: 2m 28s	remaining: 2m 46s
471:	learn: 0.4670701	total: 2m 28s	remaining: 2m 45s
472:	learn: 0.4669371	total: 2m 28s	remaining: 2m 45s
473:	learn: 0.4667337	total: 2m 29s	remaining: 2m 45s
474:	learn: 0.4664486	total: 2m 29s	remaining: 2m 45s
475:	learn: 0.4661033	total: 2m 29s	remaining: 2m 44s
476:	learn: 0.4658167	total: 2m 29s	remaining: 2m 44s
477:	learn: 0.4655502	total: 2m 30s	remaining: 2m 44s
478:	learn: 0.4653315	total: 2m 30s	remaining: 2m 43s
479:	learn: 0.4652092	total: 2m 30s	remaining: 2m 43s
480:	learn: 0.4650258	total: 2m 31s	remaining: 2m 43s
481:	learn: 0.4648636	total: 2m 31s	remaining: 2m 42s
482:	learn: 0.4647983	total: 2m 31s	remaining: 2m 42s
483:	learn: 0.4646270	total: 2m 32s	remaining: 2m 42s
484:	learn: 0.4644577	total:

620:	learn: 0.4433962	total: 3m 14s	remaining: 1m 59s
621:	learn: 0.4432616	total: 3m 15s	remaining: 1m 58s
622:	learn: 0.4430500	total: 3m 15s	remaining: 1m 58s
623:	learn: 0.4429632	total: 3m 15s	remaining: 1m 58s
624:	learn: 0.4428759	total: 3m 16s	remaining: 1m 57s
625:	learn: 0.4427785	total: 3m 16s	remaining: 1m 57s
626:	learn: 0.4426636	total: 3m 16s	remaining: 1m 57s
627:	learn: 0.4425572	total: 3m 17s	remaining: 1m 56s
628:	learn: 0.4424158	total: 3m 17s	remaining: 1m 56s
629:	learn: 0.4422870	total: 3m 17s	remaining: 1m 56s
630:	learn: 0.4422103	total: 3m 18s	remaining: 1m 55s
631:	learn: 0.4421274	total: 3m 18s	remaining: 1m 55s
632:	learn: 0.4420538	total: 3m 18s	remaining: 1m 55s
633:	learn: 0.4418927	total: 3m 19s	remaining: 1m 54s
634:	learn: 0.4418311	total: 3m 19s	remaining: 1m 54s
635:	learn: 0.4416598	total: 3m 19s	remaining: 1m 54s
636:	learn: 0.4415775	total: 3m 20s	remaining: 1m 54s
637:	learn: 0.4414617	total: 3m 20s	remaining: 1m 53s
638:	learn: 0.4412815	total:

773:	learn: 0.4275331	total: 4m 2s	remaining: 1m 10s
774:	learn: 0.4274794	total: 4m 3s	remaining: 1m 10s
775:	learn: 0.4274538	total: 4m 3s	remaining: 1m 10s
776:	learn: 0.4274229	total: 4m 3s	remaining: 1m 9s
777:	learn: 0.4272576	total: 4m 4s	remaining: 1m 9s
778:	learn: 0.4270690	total: 4m 4s	remaining: 1m 9s
779:	learn: 0.4269788	total: 4m 4s	remaining: 1m 9s
780:	learn: 0.4268714	total: 4m 5s	remaining: 1m 8s
781:	learn: 0.4268001	total: 4m 5s	remaining: 1m 8s
782:	learn: 0.4266987	total: 4m 5s	remaining: 1m 8s
783:	learn: 0.4266052	total: 4m 5s	remaining: 1m 7s
784:	learn: 0.4265548	total: 4m 6s	remaining: 1m 7s
785:	learn: 0.4264535	total: 4m 6s	remaining: 1m 7s
786:	learn: 0.4263074	total: 4m 6s	remaining: 1m 6s
787:	learn: 0.4261856	total: 4m 7s	remaining: 1m 6s
788:	learn: 0.4261536	total: 4m 7s	remaining: 1m 6s
789:	learn: 0.4260496	total: 4m 7s	remaining: 1m 5s
790:	learn: 0.4258804	total: 4m 8s	remaining: 1m 5s
791:	learn: 0.4258098	total: 4m 8s	remaining: 1m 5s
792:	lear

929:	learn: 0.4146631	total: 4m 51s	remaining: 22s
930:	learn: 0.4145482	total: 4m 52s	remaining: 21.6s
931:	learn: 0.4144580	total: 4m 52s	remaining: 21.3s
932:	learn: 0.4142984	total: 4m 52s	remaining: 21s
933:	learn: 0.4142391	total: 4m 52s	remaining: 20.7s
934:	learn: 0.4141877	total: 4m 53s	remaining: 20.4s
935:	learn: 0.4140686	total: 4m 53s	remaining: 20.1s
936:	learn: 0.4140437	total: 4m 53s	remaining: 19.8s
937:	learn: 0.4140098	total: 4m 54s	remaining: 19.4s
938:	learn: 0.4138789	total: 4m 54s	remaining: 19.1s
939:	learn: 0.4138523	total: 4m 54s	remaining: 18.8s
940:	learn: 0.4137756	total: 4m 55s	remaining: 18.5s
941:	learn: 0.4136569	total: 4m 55s	remaining: 18.2s
942:	learn: 0.4135135	total: 4m 55s	remaining: 17.9s
943:	learn: 0.4134784	total: 4m 56s	remaining: 17.6s
944:	learn: 0.4134276	total: 4m 56s	remaining: 17.3s
945:	learn: 0.4133733	total: 4m 56s	remaining: 16.9s
946:	learn: 0.4133513	total: 4m 57s	remaining: 16.6s
947:	learn: 0.4133348	total: 4m 57s	remaining: 16.

In [52]:
cb_model.score(X_test_minmaxscaled,y_test_minmaxscaled)

0.8565964149103727

In [53]:
y_pred_prob_cb_minmaxscaled = cb_model.predict_proba(X_test_minmaxscaled)
log_loss(y_true=y_test_minmaxscaled, y_pred=y_pred_prob_cb_minmaxscaled)

0.4305520118888065

In [54]:
# MaxAbsScaler
from sklearn.preprocessing import MaxAbsScaler
maxabsscaler = MaxAbsScaler()
print(maxabsscaler.fit(train_for_scaling))
train_maxabsscaled = maxabsscaler.transform(train_for_scaling)
train_maxabsscaled = pd.DataFrame(train_maxabsscaled,columns=train_for_scaling.columns)
train_maxabsscaled = pd.concat([train[['type', 'fiberID']], train_maxabsscaled], axis=1) 
train_maxabsscaled #maxabsscaling과 type,fiberID 합치기

MaxAbsScaler(copy=True)


,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,601,0.000004,0.000535,0.000784,0.000797,0.000851,0.000012,0.000087,0.000993,...,0.000075,0.000826,0.000897,0.000523,0.000679,0.000867,0.000582,0.000586,0.000539,0.000812
1,QSO,788,0.000004,0.000517,0.000761,0.000779,0.000823,0.000012,0.000085,0.000962,...,0.000071,0.000800,0.000872,0.000510,0.000687,0.000819,0.000563,0.000570,0.000531,0.000787
2,QSO,427,0.000003,0.000418,0.000614,0.000628,0.000669,0.000010,0.000069,0.000780,...,0.000060,0.000645,0.000697,0.000414,0.000559,0.000679,0.000452,0.000457,0.000428,0.000641
3,QSO,864,0.000004,0.000501,0.000733,0.000747,0.000782,0.000011,0.000082,0.000927,...,0.000068,0.000770,0.000835,0.000488,0.000647,0.000792,0.000542,0.000546,0.000507,0.000751
4,STAR_RED_DWARF,612,0.000005,0.000576,0.000790,0.000734,0.000739,0.000014,0.000095,0.000997,...,0.000087,0.000864,0.000905,0.000485,0.000611,0.000948,0.000627,0.000589,0.000500,0.000705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,0.000004,0.000511,0.000738,0.000754,0.000802,0.000011,0.000084,0.000932,...,0.000070,0.000791,0.000840,0.000497,0.000658,0.000792,0.000551,0.000549,0.000515,0.000762
199987,GALAXY,362,0.000005,0.000570,0.000775,0.000760,0.000788,0.000014,0.000092,0.000953,...,0.000097,0.000857,0.000839,0.000470,0.000625,0.001026,0.000595,0.000543,0.000487,0.000711
199988,STAR_RED_DWARF,146,0.000004,0.000547,0.000748,0.000695,0.000696,0.000013,0.000090,0.000950,...,0.000087,0.000853,0.000849,0.000458,0.000579,0.000934,0.000593,0.000560,0.000473,0.000665
199989,GALAXY,85,0.000004,0.000514,0.000737,0.000745,0.000785,0.000012,0.000083,0.000911,...,0.000063,0.000696,0.000731,0.000425,0.000571,0.000728,0.000488,0.000479,0.000441,0.000651


In [55]:
X_maxabsscaled = train_maxabsscaled.drop(['type'], axis = 1)
y_maxabsscaled = train_maxabsscaled['type']
X_train_maxabsscaled, X_test_maxabsscaled, y_train_maxabsscaled, y_test_maxabsscaled = train_test_split(X_maxabsscaled, y_maxabsscaled, test_size= 0.2, random_state=42)

In [56]:
#Catboost
cb_model.fit(X_train_maxabsscaled,y_train_maxabsscaled)

0:	learn: 2.6657351	total: 325ms	remaining: 5m 24s
1:	learn: 2.4748539	total: 653ms	remaining: 5m 25s
2:	learn: 2.3311085	total: 979ms	remaining: 5m 25s
3:	learn: 2.2131228	total: 1.3s	remaining: 5m 24s
4:	learn: 2.1140204	total: 1.63s	remaining: 5m 23s
5:	learn: 2.0306243	total: 1.95s	remaining: 5m 23s
6:	learn: 1.9551078	total: 2.28s	remaining: 5m 23s
7:	learn: 1.8855908	total: 2.6s	remaining: 5m 23s
8:	learn: 1.8259695	total: 2.93s	remaining: 5m 22s
9:	learn: 1.7709078	total: 3.25s	remaining: 5m 22s
10:	learn: 1.7204255	total: 3.58s	remaining: 5m 21s
11:	learn: 1.6737515	total: 3.9s	remaining: 5m 21s
12:	learn: 1.6305277	total: 4.22s	remaining: 5m 20s
13:	learn: 1.5916672	total: 4.54s	remaining: 5m 19s
14:	learn: 1.5551641	total: 4.86s	remaining: 5m 18s
15:	learn: 1.5200552	total: 5.19s	remaining: 5m 19s
16:	learn: 1.4883391	total: 5.51s	remaining: 5m 18s
17:	learn: 1.4581531	total: 5.83s	remaining: 5m 18s
18:	learn: 1.4285160	total: 6.16s	remaining: 5m 17s
19:	learn: 1.4015247	tota

158:	learn: 0.6167572	total: 50.5s	remaining: 4m 27s
159:	learn: 0.6160344	total: 50.8s	remaining: 4m 26s
160:	learn: 0.6152140	total: 51.1s	remaining: 4m 26s
161:	learn: 0.6135107	total: 51.5s	remaining: 4m 26s
162:	learn: 0.6118995	total: 51.8s	remaining: 4m 25s
163:	learn: 0.6104338	total: 52.1s	remaining: 4m 25s
164:	learn: 0.6098465	total: 52.4s	remaining: 4m 25s
165:	learn: 0.6084283	total: 52.7s	remaining: 4m 24s
166:	learn: 0.6068630	total: 53s	remaining: 4m 24s
167:	learn: 0.6053626	total: 53.4s	remaining: 4m 24s
168:	learn: 0.6042123	total: 53.7s	remaining: 4m 23s
169:	learn: 0.6030069	total: 54s	remaining: 4m 23s
170:	learn: 0.6017429	total: 54.3s	remaining: 4m 23s
171:	learn: 0.6009585	total: 54.6s	remaining: 4m 22s
172:	learn: 0.5997490	total: 54.9s	remaining: 4m 22s
173:	learn: 0.5990320	total: 55.2s	remaining: 4m 22s
174:	learn: 0.5984987	total: 55.5s	remaining: 4m 21s
175:	learn: 0.5980935	total: 55.9s	remaining: 4m 21s
176:	learn: 0.5970037	total: 56.2s	remaining: 4m 2

312:	learn: 0.5097277	total: 1m 39s	remaining: 3m 37s
313:	learn: 0.5093758	total: 1m 39s	remaining: 3m 37s
314:	learn: 0.5089163	total: 1m 39s	remaining: 3m 37s
315:	learn: 0.5083271	total: 1m 40s	remaining: 3m 36s
316:	learn: 0.5078489	total: 1m 40s	remaining: 3m 36s
317:	learn: 0.5073735	total: 1m 40s	remaining: 3m 36s
318:	learn: 0.5071288	total: 1m 41s	remaining: 3m 36s
319:	learn: 0.5065295	total: 1m 41s	remaining: 3m 35s
320:	learn: 0.5063357	total: 1m 41s	remaining: 3m 35s
321:	learn: 0.5059403	total: 1m 42s	remaining: 3m 35s
322:	learn: 0.5056707	total: 1m 42s	remaining: 3m 34s
323:	learn: 0.5053540	total: 1m 42s	remaining: 3m 34s
324:	learn: 0.5049688	total: 1m 43s	remaining: 3m 34s
325:	learn: 0.5045211	total: 1m 43s	remaining: 3m 33s
326:	learn: 0.5039972	total: 1m 43s	remaining: 3m 33s
327:	learn: 0.5038003	total: 1m 44s	remaining: 3m 33s
328:	learn: 0.5032343	total: 1m 44s	remaining: 3m 32s
329:	learn: 0.5030645	total: 1m 44s	remaining: 3m 32s
330:	learn: 0.5026196	total:

466:	learn: 0.4665059	total: 2m 27s	remaining: 2m 48s
467:	learn: 0.4662181	total: 2m 28s	remaining: 2m 48s
468:	learn: 0.4660158	total: 2m 28s	remaining: 2m 48s
469:	learn: 0.4659616	total: 2m 28s	remaining: 2m 47s
470:	learn: 0.4658530	total: 2m 29s	remaining: 2m 47s
471:	learn: 0.4656804	total: 2m 29s	remaining: 2m 47s
472:	learn: 0.4656108	total: 2m 29s	remaining: 2m 46s
473:	learn: 0.4653684	total: 2m 30s	remaining: 2m 46s
474:	learn: 0.4650888	total: 2m 30s	remaining: 2m 46s
475:	learn: 0.4647427	total: 2m 30s	remaining: 2m 46s
476:	learn: 0.4644944	total: 2m 31s	remaining: 2m 45s
477:	learn: 0.4643024	total: 2m 31s	remaining: 2m 45s
478:	learn: 0.4640707	total: 2m 31s	remaining: 2m 45s
479:	learn: 0.4639679	total: 2m 32s	remaining: 2m 44s
480:	learn: 0.4636304	total: 2m 32s	remaining: 2m 44s
481:	learn: 0.4635026	total: 2m 32s	remaining: 2m 44s
482:	learn: 0.4633164	total: 2m 33s	remaining: 2m 43s
483:	learn: 0.4631056	total: 2m 33s	remaining: 2m 43s
484:	learn: 0.4628507	total:

620:	learn: 0.4404947	total: 3m 16s	remaining: 2m
621:	learn: 0.4403345	total: 3m 17s	remaining: 1m 59s
622:	learn: 0.4402543	total: 3m 17s	remaining: 1m 59s
623:	learn: 0.4401705	total: 3m 17s	remaining: 1m 59s
624:	learn: 0.4400855	total: 3m 17s	remaining: 1m 58s
625:	learn: 0.4400357	total: 3m 18s	remaining: 1m 58s
626:	learn: 0.4398550	total: 3m 18s	remaining: 1m 58s
627:	learn: 0.4397573	total: 3m 18s	remaining: 1m 57s
628:	learn: 0.4396133	total: 3m 19s	remaining: 1m 57s
629:	learn: 0.4394866	total: 3m 19s	remaining: 1m 57s
630:	learn: 0.4394066	total: 3m 19s	remaining: 1m 56s
631:	learn: 0.4393581	total: 3m 20s	remaining: 1m 56s
632:	learn: 0.4392790	total: 3m 20s	remaining: 1m 56s
633:	learn: 0.4392286	total: 3m 20s	remaining: 1m 55s
634:	learn: 0.4391708	total: 3m 21s	remaining: 1m 55s
635:	learn: 0.4390634	total: 3m 21s	remaining: 1m 55s
636:	learn: 0.4389376	total: 3m 21s	remaining: 1m 54s
637:	learn: 0.4388225	total: 3m 21s	remaining: 1m 54s
638:	learn: 0.4386542	total: 3m 

773:	learn: 0.4233291	total: 4m 5s	remaining: 1m 11s
774:	learn: 0.4232513	total: 4m 5s	remaining: 1m 11s
775:	learn: 0.4231130	total: 4m 5s	remaining: 1m 10s
776:	learn: 0.4229829	total: 4m 5s	remaining: 1m 10s
777:	learn: 0.4229170	total: 4m 6s	remaining: 1m 10s
778:	learn: 0.4228676	total: 4m 6s	remaining: 1m 9s
779:	learn: 0.4228122	total: 4m 6s	remaining: 1m 9s
780:	learn: 0.4227185	total: 4m 7s	remaining: 1m 9s
781:	learn: 0.4226603	total: 4m 7s	remaining: 1m 9s
782:	learn: 0.4225766	total: 4m 7s	remaining: 1m 8s
783:	learn: 0.4224295	total: 4m 8s	remaining: 1m 8s
784:	learn: 0.4223815	total: 4m 8s	remaining: 1m 8s
785:	learn: 0.4223320	total: 4m 8s	remaining: 1m 7s
786:	learn: 0.4222316	total: 4m 9s	remaining: 1m 7s
787:	learn: 0.4221855	total: 4m 9s	remaining: 1m 7s
788:	learn: 0.4221173	total: 4m 9s	remaining: 1m 6s
789:	learn: 0.4219820	total: 4m 10s	remaining: 1m 6s
790:	learn: 0.4219000	total: 4m 10s	remaining: 1m 6s
791:	learn: 0.4218739	total: 4m 10s	remaining: 1m 5s
792:

929:	learn: 0.4119483	total: 4m 53s	remaining: 22.1s
930:	learn: 0.4118347	total: 4m 54s	remaining: 21.8s
931:	learn: 0.4117961	total: 4m 54s	remaining: 21.5s
932:	learn: 0.4117471	total: 4m 54s	remaining: 21.2s
933:	learn: 0.4117159	total: 4m 55s	remaining: 20.9s
934:	learn: 0.4116726	total: 4m 55s	remaining: 20.5s
935:	learn: 0.4115906	total: 4m 55s	remaining: 20.2s
936:	learn: 0.4115127	total: 4m 56s	remaining: 19.9s
937:	learn: 0.4114772	total: 4m 56s	remaining: 19.6s
938:	learn: 0.4113470	total: 4m 56s	remaining: 19.3s
939:	learn: 0.4112482	total: 4m 57s	remaining: 19s
940:	learn: 0.4112062	total: 4m 57s	remaining: 18.6s
941:	learn: 0.4110919	total: 4m 57s	remaining: 18.3s
942:	learn: 0.4110182	total: 4m 58s	remaining: 18s
943:	learn: 0.4109418	total: 4m 58s	remaining: 17.7s
944:	learn: 0.4108543	total: 4m 58s	remaining: 17.4s
945:	learn: 0.4108184	total: 4m 58s	remaining: 17.1s
946:	learn: 0.4106967	total: 4m 59s	remaining: 16.8s
947:	learn: 0.4106552	total: 4m 59s	remaining: 16.

In [57]:
cb_model.score(X_test_maxabsscaled,y_test_maxabsscaled)

0.8580964524113103

In [58]:
y_pred_prob_cb_maxabsscaled = cb_model.predict_proba(X_test_maxabsscaled)
log_loss(y_true=y_test_maxabsscaled, y_pred=y_pred_prob_cb_maxabsscaled)

0.4277657503242839

In [60]:
print("Robust: " , log_loss(y_true=y_test_robustscaled, y_pred=y_pred_prob_cb_robustscaled))
print("Standard: " , log_loss(y_true=y_test_standardscaled, y_pred=y_pred_prob_cb_standardscaled))
print("MinMax: " , log_loss(y_true=y_test_minmaxscaled, y_pred=y_pred_prob_cb_minmaxscaled))
print("MaxAbs: " , log_loss(y_true=y_test_maxabsscaled, y_pred=y_pred_prob_cb_maxabsscaled))

Robust:  0.42774662154063664
Standard:  0.4278871022373594
MinMax:  0.4305520118888065
MaxAbs:  0.4277657503242839


위에 보이는 바와 같이 RobustScaler의 logloss 값이 가장 작은 것을 확인할 수 있고, 이를 통해서 Scaler를 사용할 시에는 RobustScaler를 쓰는게 맞다. 그렇다면 RobustScale한 데이터에 이전에 raw data로 가장 값이 좋았던 LGBM을 사용해보도록 한다.

In [3]:
!pip3 install lightgbm

  Using cached https://files.pythonhosted.org/packages/0b/9d/ddcb2f43aca194987f1a99e27edf41cf9bc39ea750c3371c2a62698c509a/lightgbm-2.3.1-py2.py3-none-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d1/48/e9fa9e252abcd1447eff6f9257636af31758a6e46fd5ce5d3c879f6907cb/scikit_learn-0.22.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl


In [11]:
import lightgbm
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)
lgbm_model.fit(X_train_robustscaled,y_train_robustscaled)
lgbm_model.score(X_test_robustscaled,y_test_robustscaled)

0.8754718867971699

In [14]:
from sklearn.metrics import log_loss
y_pred_prob_lgbm_robustscaled = lgbm_model.predict_proba(X_test_robustscaled)
log_loss(y_true=y_test_robustscaled, y_pred=y_pred_prob_lgbm_robustscaled) #애매한데... 한번 뽑아봐야지;

0.38735228345368095

train 데이터 전체를 쪼개지말고 학습시킨다.

In [16]:
import lightgbm
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)
lgbm_model.fit(X_robustscaled,y_robustscaled)

LGBMClassifier(bagging_fraction=0.9, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, feature_fraction=0.9,
        importance_type='split', learning_rate=0.1, max_depth=15,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=2000, n_jobs=-1, num_leaves=10, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.2, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
from sklearn.model_selection import train_test_split
test = pd.read_csv('test.csv', index_col = 'id')
test

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,23.817399,22.508963,20.981106,18.517316,17.076079,25.053890,23.167848,21.335901,18.835858,...,22.246697,22.796239,21.195315,18.584486,17.154284,25.391534,22.499435,21.011918,18.499341,17.091474
199992,386,22.806983,21.937111,20.335770,20.000512,19.527369,22.498565,22.186000,20.618879,20.301204,...,21.729831,21.837511,20.196128,19.967204,19.683671,22.475338,21.853442,20.173169,19.796757,19.567372
199993,232,21.024250,19.235669,18.304061,17.808608,17.380113,21.205546,19.439533,18.344433,17.909690,...,20.722629,18.710223,17.611851,17.158519,16.843986,20.579314,18.653338,17.562108,17.120529,16.708748
199994,557,20.503424,20.286261,20.197204,20.162419,20.059832,20.976132,20.611498,20.567262,20.479318,...,20.329269,20.385262,20.129157,20.206574,20.212342,20.479879,20.280943,20.150499,20.206221,20.092909
199995,75,24.244851,22.668237,21.239333,19.284777,18.235939,25.681860,22.935289,21.642456,19.624926,...,22.308298,22.957496,21.285033,19.299120,18.307526,25.489360,22.857290,21.191862,19.237964,18.280368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,389,19.765035,19.542406,19.439286,19.357957,19.357305,20.093308,19.827172,19.714033,19.686861,...,19.882728,19.549257,19.457564,19.303361,19.483389,19.792181,19.536518,19.432463,19.376445,19.354976
209996,247,19.960030,19.609379,19.325949,19.391290,19.258862,20.319914,19.928523,19.696262,19.763771,...,20.085454,19.635609,19.381924,19.460517,19.390865,20.066552,19.604800,19.334113,19.401240,19.159412
209997,941,21.221899,20.276182,20.090775,19.927208,19.766732,21.254454,20.551190,20.365800,20.200872,...,20.998020,20.315201,20.236725,20.058801,19.988363,21.045501,20.255485,20.141549,19.906014,19.953932


In [36]:
id_for_index = test['id']
id_for_index

0        199991
1        199992
2        199993
3        199994
4        199995
          ...  
10004    209995
10005    209996
10006    209997
10007    209998
10008    209999
Name: id, Length: 10009, dtype: int64

In [48]:
from sklearn.preprocessing import RobustScaler
test_for_scaling = test.iloc[:,1:] #fiberID 제외하고 데이터 생성
print(robustScaler.fit(test_for_scaling))
test_robustscaled = robustScaler.transform(test_for_scaling) #robustscaling 적용
test_robustscaled = pd.DataFrame(test_robustscaled,index = id_for_index, columns=test_for_scaling.columns)
test_robustscaled

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)


,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,
199991,1.182702,1.048251,0.626485,-0.220330,-0.618852,1.672996,1.233289,0.682468,-0.142070,-0.504366,0.734331,1.099892,0.676638,-0.033574,-0.338060,1.827431,1.014195,0.626034,-0.045304,-0.323146
199992,0.779387,0.817574,0.363304,0.413028,0.380063,0.607737,0.842195,0.395966,0.461799,0.530203,0.531978,0.771724,0.338297,0.431661,0.502168,0.757070,0.792720,0.343134,0.387349,0.473419
199993,0.067795,-0.272150,-0.465267,-0.522964,-0.494957,0.068706,-0.251791,-0.512839,-0.523744,-0.463003,0.137658,-0.298732,-0.536779,-0.513360,-0.441137,0.061153,-0.304420,-0.537547,-0.505100,-0.446280
199994,-0.140096,0.151644,0.306794,0.482166,0.597045,-0.026932,0.215032,0.375341,0.535200,0.628807,-0.016343,0.274626,0.315620,0.512201,0.677785,0.024657,0.253596,0.335487,0.523894,0.642499
199995,1.353323,1.112500,0.731795,0.107394,-0.146202,1.934783,1.140655,0.804959,0.183104,-0.043074,0.758448,1.155090,0.707018,0.206875,0.045031,1.863337,1.136884,0.686727,0.201007,0.059354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,-0.434830,-0.148417,-0.002300,0.138643,0.310761,-0.394962,-0.097385,0.034415,0.208628,0.442365,-0.191164,-0.011535,0.088208,0.208302,0.435637,-0.227756,-0.001626,0.093302,0.247187,0.405085
209996,-0.356996,-0.121401,-0.048521,0.152877,0.270645,-0.300494,-0.057014,0.027314,0.240323,0.281907,-0.111797,0.018023,0.062595,0.261179,0.404902,-0.127051,0.021784,0.060130,0.255455,0.342167
209997,0.146688,0.147578,0.263390,0.381726,0.477605,0.089095,0.191009,0.294843,0.420452,0.490485,0.245474,0.250644,0.352044,0.462480,0.603382,0.232262,0.244868,0.332468,0.423784,0.597786


In [49]:
test_robustscaled = pd.concat([test[['fiberID']], test_robustscaled], axis=1) 
test_robustscaled #standardscaling과 type,fiberID 합치기

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,1.182702,1.048251,0.626485,-0.220330,-0.618852,1.672996,1.233289,0.682468,-0.142070,...,0.734331,1.099892,0.676638,-0.033574,-0.338060,1.827431,1.014195,0.626034,-0.045304,-0.323146
199992,386,0.779387,0.817574,0.363304,0.413028,0.380063,0.607737,0.842195,0.395966,0.461799,...,0.531978,0.771724,0.338297,0.431661,0.502168,0.757070,0.792720,0.343134,0.387349,0.473419
199993,232,0.067795,-0.272150,-0.465267,-0.522964,-0.494957,0.068706,-0.251791,-0.512839,-0.523744,...,0.137658,-0.298732,-0.536779,-0.513360,-0.441137,0.061153,-0.304420,-0.537547,-0.505100,-0.446280
199994,557,-0.140096,0.151644,0.306794,0.482166,0.597045,-0.026932,0.215032,0.375341,0.535200,...,-0.016343,0.274626,0.315620,0.512201,0.677785,0.024657,0.253596,0.335487,0.523894,0.642499
199995,75,1.353323,1.112500,0.731795,0.107394,-0.146202,1.934783,1.140655,0.804959,0.183104,...,0.758448,1.155090,0.707018,0.206875,0.045031,1.863337,1.136884,0.686727,0.201007,0.059354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,389,-0.434830,-0.148417,-0.002300,0.138643,0.310761,-0.394962,-0.097385,0.034415,0.208628,...,-0.191164,-0.011535,0.088208,0.208302,0.435637,-0.227756,-0.001626,0.093302,0.247187,0.405085
209996,247,-0.356996,-0.121401,-0.048521,0.152877,0.270645,-0.300494,-0.057014,0.027314,0.240323,...,-0.111797,0.018023,0.062595,0.261179,0.404902,-0.127051,0.021784,0.060130,0.255455,0.342167
209997,941,0.146688,0.147578,0.263390,0.381726,0.477605,0.089095,0.191009,0.294843,0.420452,...,0.245474,0.250644,0.352044,0.462480,0.603382,0.232262,0.244868,0.332468,0.423784,0.597786


In [50]:
test_pred_prob_lgbm_robustscaled = lgbm_model.predict_proba(test_robustscaled)
test_pred_prob_lgbm_robustscaled

array([[2.52680623e-07, 8.14648167e-04, 3.14476540e-09, ...,
        9.89300059e-01, 4.43855425e-08, 1.05527266e-09],
       [4.50239979e-04, 4.39441969e-01, 6.89400087e-07, ...,
        7.52954509e-11, 3.09331355e-08, 3.56854368e-06],
       [9.97797569e-01, 1.09347657e-03, 1.35452394e-08, ...,
        7.96969119e-09, 7.93067817e-12, 1.39254853e-05],
       ...,
       [1.23954350e-09, 9.99998238e-01, 6.12515287e-11, ...,
        1.41150972e-15, 4.07235263e-17, 7.86384989e-09],
       [1.32740307e-09, 9.99999849e-01, 6.85006467e-11, ...,
        4.78742658e-11, 9.69860942e-13, 1.30172496e-11],
       [4.48477389e-06, 2.16701839e-04, 4.15264983e-09, ...,
        1.20071083e-09, 2.98527357e-10, 1.09182867e-05]])

In [51]:
result = pd.DataFrame(data=test_pred_prob_lgbm_robustscaled, index=id_for_index, columns=lgbm_model.classes_)
result.head()

,GALAXY,QSO,REDDEN_STD,ROSAT_D,SERENDIPITY_BLUE,SERENDIPITY_DISTANT,SERENDIPITY_FIRST,SERENDIPITY_MANUAL,SERENDIPITY_RED,SKY,SPECTROPHOTO_STD,STAR_BHB,STAR_BROWN_DWARF,STAR_CARBON,STAR_CATY_VAR,STAR_PN,STAR_RED_DWARF,STAR_SUB_DWARF,STAR_WHITE_DWARF
id,,,,,,,,,,,,,,,,,,,
199991,2.526806e-07,0.000815,3.144765e-09,0.003322,6.177048e-06,9.890180e-07,3.376736e-07,1.951948e-07,6.534118e-03,1.719381e-10,2.642275e-08,1.991550e-10,2.092222e-05,2.704228e-10,9.866258e-08,1.098053e-09,9.893001e-01,4.438554e-08,1.055273e-09
199992,4.502400e-04,0.439442,6.894001e-07,0.248206,1.295881e-05,1.648560e-04,3.117094e-01,1.496659e-09,5.085091e-06,4.396509e-09,2.748863e-07,2.929060e-06,1.446505e-12,2.798202e-07,1.509681e-06,2.646153e-08,7.529545e-11,3.093314e-08,3.568544e-06
199993,9.977976e-01,0.001093,1.354524e-08,0.001061,5.221759e-07,1.505503e-08,3.233644e-05,1.000592e-08,8.089307e-09,8.643641e-11,2.200611e-08,1.123556e-07,6.990099e-10,7.467833e-07,7.610866e-08,2.591023e-10,7.969691e-09,7.930678e-12,1.392549e-05
199994,3.821011e-06,0.080427,2.055203e-07,0.016202,8.878275e-01,9.391829e-04,1.459596e-02,9.253896e-09,4.711761e-08,1.549288e-09,2.255684e-07,1.137674e-08,5.549410e-12,1.194185e-08,2.165658e-06,2.722048e-08,4.565533e-11,8.086298e-13,2.073683e-06
199995,2.842709e-07,0.000036,5.709477e-08,0.000178,3.168387e-08,2.338126e-09,2.781154e-07,2.016686e-09,1.335592e-06,3.002334e-11,1.085208e-09,3.823127e-10,4.120285e-10,7.516055e-09,2.595735e-12,8.976070e-11,9.997841e-01,5.065992e-11,5.419734e-10


In [55]:
sample = pd.read_csv("sample_submission.csv")
col_order = sample.columns[1:]
result = result[col_order]
result

,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
id,,,,,,,,,,,,,,,,,,,
199991,1.055273e-09,9.866258e-08,2.092222e-05,6.534118e-03,3.144765e-09,1.991550e-10,2.526806e-07,9.890180e-07,0.000815,1.719381e-10,9.893001e-01,3.322126e-03,1.098053e-09,3.376736e-07,2.704228e-10,2.642275e-08,4.438554e-08,1.951948e-07,6.177048e-06
199992,3.568544e-06,1.509681e-06,1.446505e-12,5.085091e-06,6.894001e-07,2.929060e-06,4.502400e-04,1.648560e-04,0.439442,4.396509e-09,7.529545e-11,2.482061e-01,2.646153e-08,3.117094e-01,2.798202e-07,2.748863e-07,3.093314e-08,1.496659e-09,1.295881e-05
199993,1.392549e-05,7.610866e-08,6.990099e-10,8.089307e-09,1.354524e-08,1.123556e-07,9.977976e-01,1.505503e-08,0.001093,8.643641e-11,7.969691e-09,1.061157e-03,2.591023e-10,3.233644e-05,7.467833e-07,2.200611e-08,7.930678e-12,1.000592e-08,5.221759e-07
199994,2.073683e-06,2.165658e-06,5.549410e-12,4.711761e-08,2.055203e-07,1.137674e-08,3.821011e-06,9.391829e-04,0.080427,1.549288e-09,4.565533e-11,1.620217e-02,2.722048e-08,1.459596e-02,1.194185e-08,2.255684e-07,8.086298e-13,9.253896e-09,8.878275e-01
199995,5.419734e-10,2.595735e-12,4.120285e-10,1.335592e-06,5.709477e-08,3.823127e-10,2.842709e-07,2.338126e-09,0.000036,3.002334e-11,9.997841e-01,1.779393e-04,8.976070e-11,2.781154e-07,7.516055e-09,1.085208e-09,5.065992e-11,2.016686e-09,3.168387e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,2.504459e-02,1.013760e-02,3.844393e-14,3.799174e-07,2.506878e-07,1.935463e-04,4.583914e-05,2.683162e-03,0.127772,2.442595e-09,1.916340e-11,7.742996e-02,8.617845e-09,2.775705e-02,1.308911e-09,1.541456e-07,1.675514e-16,1.272005e-09,7.289356e-01
209996,5.271476e-02,1.286734e-02,6.392837e-13,9.307596e-08,1.053544e-06,2.772169e-05,6.397162e-05,5.415868e-03,0.257253,3.015463e-09,2.825614e-11,5.729360e-02,7.433247e-09,5.941555e-02,3.217521e-09,3.996223e-07,4.793808e-15,2.281300e-09,5.549465e-01
209997,7.863850e-09,4.071145e-09,1.894793e-17,4.687877e-11,6.125153e-11,4.731426e-11,1.239543e-09,2.462169e-09,0.999998,1.335954e-13,1.411510e-15,7.810906e-07,4.129554e-12,5.843940e-07,6.706836e-14,7.050687e-13,4.072353e-17,3.407424e-13,3.811264e-07


In [57]:
result.to_csv('robustscaler_lgbm.csv',sep=',')